# AUTOMATIC1111's Stable Diffusion WebUI

https://github.com/AUTOMATIC1111/stable-diffusion-webui

Loosely based on https://colab.research.google.com/drive/1kw3egmSn-KgWsikYvOMjJkVDsPLjEMzl

**Guides**
- [Getting started on Paperspace](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/Docs/Paperspace%20Guide%20for%20Retards.md)
- [Using the WebUI](https://rentry.org/voldy)
- [Using the Inpainter](https://rentry.org/drfar)
- [Textual Inversion](https://rentry.org/aikgx)
- [Crowd-Sourced Prompts](https://lexica.art/)
- [Artist Name Prompts](https://sgreens.notion.site/sgreens/4ca6f4e229e24da6845b6d49e6b08ae7?v=fdf861d1c65d456e98904fe3f3670bd3)
- [Stable Diffusion Models](https://cyberes.github.io/stable-diffusion-models)
- [Textual Inversion Models](https://cyberes.github.io/stable-diffusion-textual-inversion-models/)
- [Have I Been Trained?](https://haveibeentrained.com/)

# Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded, dependencies will be auto-updated.

**Where to store the models**

`/storage/` is persistent storage shared across all machines on your account. Mounted to your machine.

`/notebooks/` is storage for this notebook only. This directory has to be copied into your machine which can increase start/stop times if the directory is very large. To avoid this, put large files in `/storage/`.

`/tmp/` <mark style="background-color:lime">is not a persistent directory, meaning your files there will be deleted when the machine turns off.</mark>

<br>

<mark style="background-color: #ff780082">If you are having storage issues</mark>, set `repo_storage_dir` to `/tmp/stable-diffusion`.

<br><br>

<mark>You must uncomment the correct section and run the block below or else the notebook won't work!</mark>

Select the section you want and do `ctrl + /` to uncomment.

Make sure there isn't a space in front of any variable. Don't put a trailing slash on directory paths. If you change any settings here, rerun this cell.

In [ ]:
import os
# Free tier
# symlink_to_notebooks = True                          # Enables the creation of symlinks back to /notebooks/
# model_storage_dir = '/tmp/stable-diffusion/models'   # Where the models will be downloaded to.
# repo_storage_dir = '/notebooks'                      # Where the repository will be downloaded to.

# Paid Tier
symlink_to_notebooks = False
model_storage_dir = '/models'
repo_storage_dir = '/code'


activate_s3 = False
remoteDir = ''
bucketName = '' # Must fill in
basePath = '/notebooks/outputs/'
storageQuota = 10000 * 0.9  # 90% of 10GB
os.environ['S3_HOST_URL'] = ''
os.environ['S3_ACCESS_KEY'] = '' # Must fill in
os.environ['S3_SECRET_KEY'] = '' # Must fill in
os.environ['S3_REGION'] = ''
# Other optional settings
# You don't have to change these if you don't want to

activate_xformers = False                              # Enables the xformers optimizations using pre-built wheels.
                                                       # Setting to True will automatically set up your environment/machine for xformers. 

link_novelai_anime_vae = True                         # Enables the linking of animevae.pt to each of the NovelAI models.
                                                       # Set to True if you've downloaded both the NovelAI models and hypernetworks.

download_scripts = True                               # Download custom scripts? Only reason why you would leave it disabled is because it may
                                                       # take a while to complete.

activate_deepdanbooru = True                          # Enable and install DeepDanbooru -> https://github.com/KichangKim/DeepDanbooru

activate_medvram = False                                # Enable medvram option.
                                                       # These are model optimizations which will reduce VRAM usage at the expense of some speed.
                                                       # Set to False if you have a lot of VRAM.

disable_pickle_check = False                           # Disable the automatic check for unexpected data in model files.
                                                       # Leave this set to False unless you have a reason to disable the check.

gradio_port = False                                    # Launch Gradio on a specific port. Set to False to let Gradio choose a port.
                                                       # This disables online Gradio app mode and you will only be able to access it on your local network.

gradio_auth = False                                    # Enable gradio_auth and insecure-extension-access option.
                                                       # Set to "me:password" to let Gradio auth.

search_paperspace_datasets = True                      # Enable searching for checkpoints in /datasets to link to the webui

ui_theme = None                                      # Set the WEB UI theme. Values can be None (default) or 'dark'.

# ===============================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store symlink_to_notebooks model_storage_dir repo_storage_dir activate_xformers link_novelai_anime_vae download_scripts activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth search_paperspace_datasets ui_theme

## Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [ ]:
%store -r model_storage_dir repo_storage_dir link_novelai_anime_vae search_paperspace_datasets
import os
import glob
import subprocess

def delete_broken_symlinks(dir):
    deleted = False
    for file in os.listdir(dir):
        path = f'{dir}/{file}'
        if os.path.islink(path) and not os.path.exists(os.readlink(path)):
            print(f'Symlink broken, removing: {file}')
            os.unlink(path)
            deleted = True
    if deleted:
        print('')
    
def symlink_models(source_dir, filetype, destination_dir):
    # don't pass directory paths with trailing slash
    if not os.path.isdir(source_dir):
        print('Not a directory:', source_dir)
        return
    for file in os.listdir(source_dir):
        if file.endswith(filetype):
            path = f'{destination_dir}/{file}'
            if not os.path.exists(path):
                print(f'New model: {file}')
                !ln -s "{source_dir}/{file}" "{destination_dir}/{file}"
            !ls -la --block-size=GB "{destination_dir}/{file}"

# Check for broken symlinks and remove them
delete_broken_symlinks(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')
delete_broken_symlinks(f'{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks')

# Link models
symlink_models(model_storage_dir, 'ckpt', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link models one level recursive
# You can organize your models in your model storage dir into sub-folders.
for file in os.listdir(model_storage_dir):
    if os.path.isdir(os.path.join(model_storage_dir, file)) and file != 'hypernetworks':
        symlink_models(os.path.join(model_storage_dir, file), 'ckpt', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link config yaml files
symlink_models(model_storage_dir, 'yaml', f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion')

# Link hypernetworks
symlink_models(f'{model_storage_dir}/hypernetworks', 'pt', f'{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks')

# Link the NovelAI files for each of the NovelAI models
os.chdir(f'{model_storage_dir}')
for model in glob.glob('novelai-*.ckpt'):
    yaml = model.replace('.ckpt', '.yaml')
    if os.path.exists(yaml) and not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{yaml}'):        
        print(f'New NovelAI model config: {yaml}')
        !ln -s "{model_storage_dir}/{yaml}" "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{yaml}"
    !ls -la --block-size=GB "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{yaml}"

if link_novelai_anime_vae:
    for model in glob.glob('novelai-*.ckpt'):
        if os.path.exists(f'{model_storage_dir}/hypernetworks/animevae.pt'):
            vae = model.replace('.ckpt', '.vae.pt')
            if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{vae}'):
                print(f'Linking NovelAI {vae} and {model}')
                !ln -s "{model_storage_dir}/hypernetworks/animevae.pt" "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{vae}"
            !ls -la --block-size=GB "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/{vae}"
        else:
            print(f'{model_storage_dir}/hypernetworks/animevae.pt NOT FOUND')
            

if search_paperspace_datasets:
    print('Searching /datasets')
    # Have to use subprocess for this
    s = subprocess.getoutput(f'find /datasets -name "*.ckpt" -print -exec ln -s "{{}}" "{repo_storage_dir}/stable-diffusion-webui/models/Stable-diffusion/" \;')
    print(s)

# Launch the WebUI

Run this block to launch the WebUI. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

See [shared.py](https://github.com/AUTOMATIC1111/stable-diffusion-webui/blob/master/modules/shared.py#L22) to view the code for the launch args. There's a lot of good info in there about exactly what the args do. If you aren't a programmer, [here's the wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings).

#### Troubleshooting
- If you have any issues, try restarting the kernel.
- `EOFError: Ran out of input` probably means you ran out of storage space and the model `.ckpt` file wasn't downloaded completely. Try cleaning up your files. There are some helpful scripts in the Tools section below.
- `The file may be malicious, so the program is not going to read it` means the program encountered unexpected data in the model file (the technical term is "pickle"). Merging models can cause this. You can disable this feature by setting `disable_pickle_check` to True in the settings block.
- Try updating your notebook using the block in the Tools section below.
- If you're still having issues, delete `stable-diffusion-webui` and reinstall.

In [ ]:
%store -r model_storage_dir repo_storage_dir activate_xformers activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth ui_theme
%cd "{repo_storage_dir}/stable-diffusion-webui"

import utils
# Enable optional args automatically based on settings
x_arg = '--xformers' if activate_xformers else ''
dd_arg = '--deepdanbooru' if activate_deepdanbooru else ''
mvram_arg = '--medvram' if activate_medvram else ''
pickled = '--disable-safe-unpickle' if disable_pickle_check else ''
port = f'--port {gradio_port}' if gradio_port else '--share'
auth = f'--gradio-auth {gradio_auth} --enable-insecure-extension-access' if gradio_auth else ''
theme = f'--theme {ui_theme}' if ui_theme else ''
observer = utils.start_watching(bucketName=bucketName, outputPath=basePath, remoteDir=remoteDir, bucketQuota=storageQuota) if activate_s3 else ''

# Launch args go below:
!python webui.py {x_arg} {dd_arg} {mvram_arg} {pickled} {port} {auth} {theme} --gradio-debug